In [11]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import pandas as pd
import numpy as np
import mne
from mne.decoding import CSP
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization, Activation, 
                                    DepthwiseConv2D, SeparableConv2D, Dense, 
                                    AveragePooling2D, Dropout, Flatten)
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import tensorflow as tf
import random

# Set seeds for reproducibility
def set_seeds(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
set_seeds()

# ==============================================================================
# CONFIGURATION (UPDATED TO REDUCE OVERFITTING)
# ==============================================================================
MTC_BASE_PATH = '/kaggle/input/mtcaic3'
ARTIFACT_Z_SCORE_THRESHOLD = 5.0
MOTION_THRESHOLD = 7.7966 
EEG_CHANNELS = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
CSP_COMPONENTS = 4
EEGNET_F1 = 8
EEGNET_D = 2
EEGNET_F2 = 16
DROPOUT_RATE = 0.6  # Increased dropout
KERNEL_LENGTH = 64
EPOCHS = 100
BATCH_SIZE = 32
L2_REG = 0.001  # L2 regularization
AUGMENT_FACTOR = 1.2  # Data augmentation factor

# ==============================================================================
# PREPROCESSING FUNCTIONS
# ==============================================================================
def load_mtc_mi_trial_data(row, base_path):
    """Loads a single MI trial's RAW EEG data."""
    id_num = row['id']
    if 4800 < id_num <= 4900: split = 'validation'
    elif id_num > 4900: split = 'test'
    else: split = 'train'
    
    eeg_path = os.path.join(base_path, 'MI', split, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
    if not os.path.exists(eeg_path): return None
    
    eeg_data = pd.read_csv(eeg_path)
    trial_num = int(row['trial'])
    samples_per_trial = 2250
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    return eeg_data.iloc[start_idx:end_idx]

def check_for_motion_artifact(trial_df, threshold):
    """Checks for excessive motion in a trial."""
    motion_sensors = ['AccX', 'AccY', 'AccZ', 'Gyro1', 'Gyro2', 'Gyro3']
    if trial_df[motion_sensors].std().max() > threshold:
        return True
    return False

def check_for_zscore_artifact_single_trial(trial_df, z_threshold):
    """Checks for statistical artifacts in a trial."""
    for channel in EEG_CHANNELS:
        if channel in trial_df.columns:
            data = trial_df[channel].values
            if np.std(data) < 1e-8: continue
            z_scores = (data - np.mean(data)) / np.std(data)
            if np.any(np.abs(z_scores) > z_threshold):
                return True
    return False

def run_ica_on_subject(subject_df):
    """Applies ICA to a subject's entire dataset to remove artifacts."""
    sfreq = 250
    
    info = mne.create_info(
        ch_names=EEG_CHANNELS, 
        sfreq=sfreq, 
        ch_types='eeg',
        verbose=False
    )
    
    # Convert to volts (μV to V)
    raw = mne.io.RawArray(
        subject_df[EEG_CHANNELS].T.values * 1e-6, 
        info, 
        verbose=False
    )
    
    # Apply bandpass filter (1-40 Hz)
    raw.filter(
        l_freq=1.0, 
        h_freq=40.0, 
        fir_design='firwin', 
        verbose=False
    )

    # Perform ICA
    ica = mne.preprocessing.ICA(
        n_components=8, 
        random_state=97, 
        max_iter='auto', 
        verbose=False
    )
    ica.fit(raw)
    
    # Automatically detect and exclude EOG artifacts
    eog_indices, _ = ica.find_bads_eog(raw, ch_name='FZ')
    ica.exclude.extend(eog_indices)
    
    # Apply ICA cleaning
    ica.apply(raw, verbose=False)
    
    # Return cleaned data in μV
    cleaned_data = raw.get_data() * 1e6
    return pd.DataFrame(cleaned_data.T, columns=EEG_CHANNELS)

def preprocess_data():
    """Full preprocessing pipeline with fixed label handling."""
    print("Loading data index files...")
    train_df = pd.read_csv(os.path.join(MTC_BASE_PATH, 'train.csv'))
    validation_df = pd.read_csv(os.path.join(MTC_BASE_PATH, 'validation.csv'))
    test_df = pd.read_csv(os.path.join(MTC_BASE_PATH, 'test.csv'))
    
    # Add split identifiers
    train_df['split'] = 'train'
    validation_df['split'] = 'validation'
    test_df['split'] = 'test'
    
    # Combine and filter for MI tasks
    all_trials_index = pd.concat([train_df, validation_df, test_df], ignore_index=True)
    mi_trials_index = all_trials_index[all_trials_index['task'] == 'MI'].copy()
    
    # Artifact rejection
    print("Performing artifact rejection...")
    artifact_ids_to_drop = []
    for i, row in mi_trials_index.iterrows():
        if (i+1) % 250 == 0: 
            print(f"  Processed {i+1}/{len(mi_trials_index)} trials...")
            
        raw_trial = load_mtc_mi_trial_data(row, MTC_BASE_PATH)
        if raw_trial is None: 
            artifact_ids_to_drop.append(row['id'])
            continue
            
        if (check_for_motion_artifact(raw_trial, MOTION_THRESHOLD) or
            check_for_zscore_artifact_single_trial(raw_trial, ARTIFACT_Z_SCORE_THRESHOLD)):
            artifact_ids_to_drop.append(row['id'])

    print(f"Removed {len(artifact_ids_to_drop)} trials due to artifacts")
    clean_index = mi_trials_index[~mi_trials_index['id'].isin(artifact_ids_to_drop)]
    
    # Store labels and IDs before grouping
    labels = {split: [] for split in ["train", "validation", "test"]}
    trial_ids = {split: [] for split in ["train", "validation", "test"]}
    
    # Group data by subject for ICA processing
    print("Grouping data by subject for ICA processing...")
    grouped_data = {split: {} for split in ["train", "validation", "test"]}
    for _, row in clean_index.iterrows():
        trial_data = load_mtc_mi_trial_data(row, MTC_BASE_PATH)
        if trial_data is not None:
            split = row['split']
            subject = row['subject_id']
            
            # Store label and ID immediately
            labels[split].append(row['label'])
            trial_ids[split].append(row['id'])
            
            if subject not in grouped_data[split]: 
                grouped_data[split][subject] = []
            grouped_data[split][subject].append(trial_data)

    # Process each subject with ICA
    print("Applying ICA cleaning...")
    ica_cleaned_trials = {split: [] for split in ["train", "validation", "test"]}
    
    for split_name, subjects in grouped_data.items():
        print(f"  Processing {split_name} set...")
        for subject, trials in subjects.items():
            if not trials: 
                continue
                
            # Combine trials for the subject
            subject_df = pd.concat(trials, ignore_index=True)
            
            # Center data by subtracting subject mean
            subject_means = subject_df[EEG_CHANNELS].mean()
            centered_subject_df = subject_df.copy()
            centered_subject_df[EEG_CHANNELS] = centered_subject_df[EEG_CHANNELS] - subject_means
            
            # Apply ICA
            ica_cleaned_subject_df = run_ica_on_subject(centered_subject_df)
            
            # Split back into individual trials
            for i in range(len(trials)):
                start = i * 2250
                end = start + 2250
                trial_data = ica_cleaned_subject_df.iloc[start:end]
                ica_cleaned_trials[split_name].append(trial_data[EEG_CHANNELS].values)
    
    # Verify label consistency
    for split in ["train", "validation", "test"]:
        assert len(ica_cleaned_trials[split]) == len(labels[split]), \
            f"Mismatch in {split} set: {len(ica_cleaned_trials[split])} trials vs {len(labels[split])} labels"
    
    print("Label consistency verified across all splits")
    return ica_cleaned_trials, labels, trial_ids

# ==============================================================================
# DATA AUGMENTATION
# ==============================================================================
def augment_data(X, y, factor=1.2):
    """Augment EEG data with random noise and scaling."""
    augmented_X = []
    augmented_y = []
    
    for _ in range(int(factor) - 1):
        for i in range(len(X)):
            # Original data
            original = X[i]
            
            # Add Gaussian noise
            noise = np.random.normal(0, 0.05, original.shape)
            augmented = original + noise
            
            # Random amplitude scaling
            scale = np.random.uniform(0.9, 1.1)
            augmented = augmented * scale
            
            augmented_X.append(augmented)
            augmented_y.append(y[i])
    
    # Combine original and augmented data
    X_aug = np.concatenate([X] + augmented_X)
    y_aug = np.concatenate([y] + augmented_y)
    
    # Shuffle the augmented dataset
    indices = np.arange(X_aug.shape[0])
    np.random.shuffle(indices)
    
    return X_aug[indices], y_aug[indices]

# ==============================================================================
# EEGNET MODEL DEFINITION (WITH REGULARIZATION)
# ==============================================================================
def create_eegnet(nb_classes, Chans=4, Samples=2250, 
                 dropoutRate=DROPOUT_RATE, kernLength=KERNEL_LENGTH, 
                 F1=EEGNET_F1, D=EEGNET_D, F2=EEGNET_F2):
    """Create the EEGNet model architecture with regularization."""
    input1 = Input(shape=(Chans, Samples, 1))

    # Block 1: Temporal convolution
    block1 = Conv2D(F1, (1, kernLength), padding='same', 
                   use_bias=False, kernel_regularizer=l2(L2_REG))(input1)
    block1 = BatchNormalization()(block1)
    
    # Depthwise convolution (spatial filtering)
    block1 = DepthwiseConv2D((Chans, 1), use_bias=False, 
                            depth_multiplier=D,
                            depthwise_constraint=max_norm(1.),
                            kernel_regularizer=l2(L2_REG))(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = Dropout(dropoutRate)(block1)

    # Block 2: Separable convolution
    block2 = SeparableConv2D(F2, (1, 16),
                           use_bias=False, padding='same',
                           kernel_regularizer=l2(L2_REG))(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = Dropout(dropoutRate)(block2)

    # Classification with regularization
    flatten = Flatten()(block2)
    dense = Dense(nb_classes, kernel_constraint=max_norm(0.25), 
                 kernel_regularizer=l2(L2_REG))(flatten)
    softmax = Activation('softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)

# ==============================================================================
# MAIN EXECUTION (WITH AUGMENTATION AND REGULARIZATION)
# ==============================================================================
def main():
    # Step 1: Preprocessing
    print("=" * 80)
    print("STEP 1: DATA PREPROCESSING")
    print("=" * 80)
    ica_cleaned_trials, labels, trial_ids = preprocess_data()
    
    # Step 2: Prepare data for CSP
    print("\n" + "=" * 80)
    print("STEP 2: CSP FEATURE EXTRACTION")
    print("=" * 80)
    
    # Format data: n_trials x n_channels x n_times
    X_train = np.array(ica_cleaned_trials['train']).transpose(0, 2, 1)
    X_val = np.array(ica_cleaned_trials['validation']).transpose(0, 2, 1)
    X_test = np.array(ica_cleaned_trials['test']).transpose(0, 2, 1)
    
    # Encode labels
    le = LabelEncoder()
    y_train = le.fit_transform(labels['train'])
    y_val = le.transform(labels['validation'])
    
    print(f"Original training set: {X_train.shape} trials")
    print(f"Validation set: {X_val.shape} trials")
    
    # Train CSP with full time-series output
    print(f"Training CSP with {CSP_COMPONENTS} components...")
    csp = CSP(n_components=CSP_COMPONENTS, reg='ledoit_wolf', 
              transform_into='csp_space', log=None)
    
    csp.fit(X_train, y_train)
    
    # Apply CSP to all datasets (returns n_trials x n_components x n_times)
    print("Applying CSP transformations...")
    X_train_csp = csp.transform(X_train)
    X_val_csp = csp.transform(X_val)
    X_test_csp = csp.transform(X_test)
    
    print(f"Shapes after CSP: Train={X_train_csp.shape}, Val={X_val_csp.shape}, Test={X_test_csp.shape}")
    
    # Data augmentation for training set
    print(f"\nAugmenting training data by factor of {AUGMENT_FACTOR}...")
    X_train_aug, y_train_aug = augment_data(X_train_csp, y_train, factor=AUGMENT_FACTOR)
    print(f"Augmented training set: {X_train_aug.shape} trials")
    
    # Reshape for EEGNet: (trials, channels, time, 1)
    X_train_eegnet = X_train_aug[..., np.newaxis]
    X_val_eegnet = X_val_csp[..., np.newaxis]
    X_test_eegnet = X_test_csp[..., np.newaxis]
    
    print(f"Shapes for EEGNet: Train={X_train_eegnet.shape}, Val={X_val_eegnet.shape}")
    
    # One-hot encode labels
    y_train_oh = to_categorical(y_train_aug)
    y_val_oh = to_categorical(y_val)
    
    # Step 3: Train EEGNet with regularization
    print("\n" + "=" * 80)
    print("STEP 3: TRAINING EEGNET MODEL WITH REGULARIZATION")
    print("=" * 80)
    
    model = create_eegnet(
        nb_classes=len(le.classes_),
        Chans=CSP_COMPONENTS,
        Samples=2250
    )
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),  # Lower learning rate
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print(model.summary())
    
    # Callbacks
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.5, 
        patience=8,  # Increased patience
        min_lr=1e-6, 
        verbose=1
    )
    
    early_stop = EarlyStopping(
        monitor='val_loss', 
        patience=20,  # Increased patience
        restore_best_weights=True, 
        verbose=1
    )
    
    checkpoint = ModelCheckpoint(
        'best_model.keras',
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=0
    )
    
    # Train model
    history = model.fit(
        X_train_eegnet, y_train_oh,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val_eegnet, y_val_oh),
        callbacks=[reduce_lr, early_stop, checkpoint],
        verbose=2
    )
    
    # Load best model weights
    model = tf.keras.models.load_model('best_model.keras')
    
    # Step 4: Generate predictions
    print("\n" + "=" * 80)
    print("STEP 4: GENERATING TEST PREDICTIONS")
    print("=" * 80)
    
    # Predict on test set
    y_test_probs = model.predict(X_test_eegnet, verbose=0)
    y_test_pred = np.argmax(y_test_probs, axis=1)
    test_labels = le.inverse_transform(y_test_pred)
    
    # Create submission file
    submission = pd.DataFrame({
        'id': trial_ids['test'],
        'label': test_labels
    })
    
    # Ensure test IDs are in correct order
    test_df = pd.read_csv(os.path.join(MTC_BASE_PATH, 'test.csv'))
    test_df = test_df[test_df['task'] == 'MI']
    submission = submission.merge(test_df[['id']], on='id', how='right').sort_values('id')
    
    # Fill any missing predictions with 'Left' as fallback
    submission['label'] = submission['label'].fillna('Left')
    
    # Save submission
    submission_file = 'submission.csv'
    submission.to_csv(submission_file, index=False)
    print(f"Saved predictions to {submission_file}")
    print(f"Submission head:\n{submission.head()}")

if __name__ == '__main__':
    main()

STEP 1: DATA PREPROCESSING
Loading data index files...
Performing artifact rejection...
  Processed 250/2500 trials...
  Processed 500/2500 trials...
  Processed 750/2500 trials...
  Processed 1000/2500 trials...
  Processed 1250/2500 trials...
  Processed 1500/2500 trials...
  Processed 1750/2500 trials...
  Processed 2000/2500 trials...
  Processed 2250/2500 trials...
Removed 234 trials due to artifacts
Grouping data by subject for ICA processing...
Applying ICA cleaning...
  Processing train set...
Fitting ICA to data using 8 channels (please be patient, this may take a while)
Selecting by number: 8 components
Fitting ICA took 0.7s.
Using EOG channel: FZ
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 d

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Fitting ICA to data using 8 channels (please be patient, this may take a while)
Selecting by number: 8 components
Fitting ICA took 0.7s.
Using EOG channel: FZ
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Fitting ICA to data using 8 channels (please be patient, this may take a while)
Selecting by number: 8 components
Fitting ICA took 0.6s.
Using EOG channel: FZ
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB

...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Fitting ICA took 0.1s.
Using EOG channel: FZ
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff f

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


    Using tolerance 7.5e+03 (2.2e-16 eps * 8 dim * 4.2e+18  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using LEDOIT_WOLF
Done.
Estimating class=1 covariance using LEDOIT_WOLF
Done.
Applying CSP transformations...
Shapes after CSP: Train=(2172, 4, 2250), Val=(45, 4, 2250), Test=(49, 4, 2250)

Augmenting training data by factor of 1.2...
Augmented training set: (2172, 4, 2250) trials
Shapes for EEGNet: Train=(2172, 4, 2250, 1), Val=(45, 4, 2250, 1)

STEP 3: TRAINING EEGNET MODEL WITH REGULARIZATION


ValueError: Unrecognized keyword arguments passed to DepthwiseConv2D: {'kernel_regularizer': <keras.src.regularizers.regularizers.L2 object at 0x7a6802322050>}